# 本文件说明

- 优浪公司项目
- 预处理及特征值计算

# 基本设置

In [12]:
##load packages, needed
# encoding=utf-8

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import pickle

import joblib
%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import os
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from pandas.io import sql
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [ ]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length])            
        return data

# 导入数据

In [5]:
pos_samples = {"pos_samples":[{"id":1,
                               "A":"银保监会：引导村镇银行的设立重点向中西部地区倾斜。",
                               "B":"筹码高位分散，调整早有预示//@中欧苹果:平安最近不安"},
                              {"id":2,
                               "A":"新华网北京7月17日消息，财政部17日晚连发4份通报，公布云南、广西、安徽、宁波等地政府违法违规举债担保问责案例www.huijindi.com。相关地方政府及时问责违法违规举债担保行为，对防范地方债务风险具有积极意义。　　涉及云南省的通报点名保山市、昆明市宜良县、楚雄州禄丰县、普洱市景东县等4个地方违法违规举债担保。以昆明市宜良县为例，2016年8月，宜良县金汇国有资产经营有限责任公司与光大兴陇信托有限责任公司签订信托融资协议，计划融资金额5亿元h+u+i+j+i+n+d+i+c+o+m。宜良县人大常委会出具决议，承诺将该笔融资资金列入县本级财政公共预算，按时足额偿还贷款本息；宜良县人民政府签订了关于“财产信托标的债权”的《债券债务确认协议》，承担无条件和不可撤销的标的债权支付义务；宜良县财政局出具将相关偿付“财产权信托标的债权”列入财政预算和中期财政预算的函。截至2017年2月底，该笔融资到位4.24亿元。整改期间，相关承诺函被撤回来自huijindi.com。　　涉及广西壮族自治区的通报点名来宾市、贺州市两起违法违规举债担保案例。记者注意到，上述两地均以市政道路、防洪堤、排洪河桥等公益性资产作为租赁物进行融资。整改期间，来宾市和贺州市政府都经协商有关方面后按程序撤回相关函件，分别将已到账的7亿元和6亿元资金退回有关方面汇_金_网。　　涉及安徽省的通报指出池州经济技术开发区以政府购买服务名义，支持企业通过应收账款抵质押融资实际替政府举债的行为。作为整改措施，池州经济技术开发区解除了相关政府购买服务协议、协商撤销《应收账款质押协议》及相关融资要件、全部归还了5亿元银行贷款。另外，交通银行池州分行由于违法违规向地方政府提供融资并要求地方政府违法违规提供担保，也受到银保监会池州银监分局相应处罚欢迎huijindi.com。　　涉及宁波市的通报指出鄞州区财政局与鄞州城投签订回购协议以建设-移交(BT)方式举债的行为。2017年5月4日，鄞州区财政局、鄞州区城投公司终止了2015年、2016年分别签订的《回购协议》。推荐阅读：增值税改革首月净减税348亿 制造业获益最大　　通报显示，除撤回承诺函、终止相关协议，上述地方还对多名负责人给予党内严重警告、行政撤职、记大过等处分www.huijindi.com。部分地市被要求向省级人民政府作出书面检查，并被全省通报。　　相关地方政府表示，将深刻汲取教训，举一反三，树立正确的政绩观，严格规范政府举债行为，积极稳妥处置隐性债务风险，坚决打赢防范化解重大风险攻坚战。推荐信息：>>> 传美图本周进行上市聆讯 或于下月24日挂牌>>> 中国楼市开年遇冷 首周各线城市成交回落>>> 大宗商品正在释放买入“信号”？>>> 持仓分析：期指主力信心不足>>> 北京首批限竞房入市现“日光”  两项目摇号不透明遭查处",
                               "B":"　　红网时刻7月1日讯（通讯员 王志平）6月29日20时41分许，京港澳高速衡阳段发生一起大型客车和半挂车相撞交通事故。截至30日中午12点26分，事故共造成18人死亡 ，14人受伤。　　事发后，在泰康人寿总公司的统筹协调下，泰康人寿湖南分公司与河南分公司两地联动，有序开展相关工作。湖南分公司连夜启动重大事件应急预案，迅速成立以分公司总经理黄开琢为组长，运营分管总张旭丽为副组长、业管部经理黄璟、衡阳中心支公司总经理肖家龙为组员的领导小组。事发地衡阳中心支公司理赔核心骨干及工作人员已赶赴现场、医院进行排查服务工作。　　泰康人寿密切关注客户可能在本次事故中出险的情况，并全力以赴妥善处置，启动理赔绿色通道服务机制，同时针对此次事故，开通5项应急服务举措：开通24小时全方位受理客户报案和咨询的热线电话95522，客户或家属也可联系其保险代理人，或可拨打泰康人寿湖南分公司理赔电话：18973438738、13974891869；启用业务办理绿色通道，减免部分理赔受理资料；放宽定点医院等限制；利用移动调查设备快速简化查勘；安排理赔专项资金，先行赔付。　　面对突如其来的灾难，泰康人寿一直“在路上、在现场、在行动”，除了在理赔方面迅速做出反应外，将安排专人对出险客户家属进行慰问，以最快的速度给客户最大的帮助。"},
                              {"id":3,
                               "A":"为顺应行业发展趋势，落实监管新政导向和满足客户日益增长的保险需求，加快公司转型和价值成长，在充分对标行业和市场，深入调研渠道和客户需求的基础上，中邮人寿保险股份有限公司特别推出“中邮年年好多多保A款保险产品计划”，该产品计划包含“中邮年年好多多保A款年金保险”，“中邮附加综合意外伤害保险”和“中邮年年好邮保百万C款医疗保险”。本产品计划具有“收益锁定 稳增长”、“百万医疗 轻松享”、“意外保障 全呵护”、“自由搭配 功能强”。 您可以通过扫码关注中邮保险官方微信,快速体验产品信息了解、保单查询、保费测算等功能。",
                               "B":"车库里车辆自燃，旁边车辆、房屋、部分货物受损！太平财产保险有限公司黑龙江分公司为什么拒赔？律师这样解答！"}]}

In [6]:
neg_samples = {"neg_samples":[{"id":4,
                               "A":"要是五点起床失败了非寿险精算我就打算写完名字就交卷",
                               "B":"_(:з」∠)_  然而我还是交不起商业保险…… ​"},
                              {"id":5,
                               "A":"祁安保险经纪主要领导与各地分公司主要负责人合影 去年5月，保险经纪牌照获保监会批复。6月，祁安保险经纪正式成立。今年3月，“蓝海祁航 诚行天下”战略发布会举行，祁安保险经纪宣告开启全国布局。   作为新明中国控股多元化发展的重要战略构成，祁安保险经纪的耀眼表现获得了集团领导以及新闻媒体的极大关注，祁安在成立运营的一年时间内取得了哪些成果，下半年以及接下来一年的经营目标又是如何安排的，今天召开的年中工作会议都一一给出了答案。   2018年6月30日，“兄弟情未来赢——荣耀在您身边”祁安保险经纪2018年中工作会议在杭州远扬大厦隆重召开，祁安保险经纪常务副总罗建灿、祁安保险经纪常务副总陈剑峰、祁安保险运营部总监郑尊义等主要领导出席会议并分别做了半年度工作总结与分享。杭州第一分公司、第二分公司、第三分公司、舟山分公司、山东分公司、河北分公司、黑龙江分公司、辽宁分公司、吉林分公司、天津分公司、北京分公司、大连分公司、湖北分公司、广州分公司、安徽分公司以及河南分公司等近五十余位主要负责人出席了会议并逐一进行了业务交流发言，其中，六家分公司现场签署了年度业绩承诺书。   业绩显著 祁安各项业务逐渐夯实   “截止今年6月，经纪费收入1381万、供应商合作签约55份、开设省级分公司7家、地市分公司7家、接受企业客户委托书32份” 会议伊始，祁安保险经纪常务副总罗建灿用一系列的精准数字汇报了公司成立以来所取得的业绩，其中，浙江分公司与杭州第二分公司都交出了600万以上营收的亮眼成绩单。下半年祁安还将计划设立省级分公司15家、地市分公司30家。随着越来越多地方分公司的设立以及业务的陆续开展，未来，祁安保险经纪费用收入也将会实现几何级倍数的提升。   祁安保险经纪常务副总罗建灿作半年度工作总结发言 此外，祁安一直在积极推进与保险公司的合作签约，截止目前，祁安已完成与包括太平财险、平安财险、中国人保等在内的共19家财险公司的合作，与中国人寿、百年人寿、昆仑健康等7家寿险公司的合作，祁安保险经纪APP也已与9家保险达成了战略合作。罗建灿表示，接下来将会积极推进与保险公司的总对总签约，从而为各分公司提供越来越多的业务支持。   同时，祁安保险经纪的内部系统建设以及品牌宣传也渐臻完善与显现，罗建灿副总经理在报告中指出，在祁安保险经纪成立至今的一年时间里已经完成了对核心业务系统、财务系统的框架搭建与测试，公司的基本法也有望在近期汇编成征求意见稿并开展测试。在品宣方面，目前祁安品牌部门已经成立并已实现了全平台宣传矩阵，上半年，通过战略发布会的举办以及微电影的制作宣传，祁安在各平台总曝光量已经达到了50万次以上，祁安的品牌影响力正逐渐得到强化。   祁安保险经纪APP上线 助力展业新武器   随着中国互联网网民越过7亿关口，互联网在人们日常生活场景中的逐渐普及与应用，互联网保险也进入了越来越多人的视野中，如何让互联网的科技创新、互联网思维更好赋能保险行业，祁安在这一年里思考了很多。   祁安保险经纪的互联网保险业务今年3月获得了保监会的审批，相应的APP开发也已处于调试与完善阶段，目前，已完成了APP安卓版的上线，IOS版本也会在近期上线测试。   祁安保险经纪常务副总陈剑峰介绍线上APP 在今天上午的会议现场，祁安保险经纪常务副总陈剑峰对APP的亮点进行了详细介绍并将其定位为中国顾问式风险管理服务商，“通过这款APP对各类风险管理场景入口及保险需求的分析与评测，可以做到为客户提供风险管理咨询、提供保险需求，避免了保险一上来就‘卖’的尴尬局面”，陈剑峰在发言中指出了目前保险行业的痛点所在，祁安保险经纪APP的上线将让消费者告别被动保险，让风险透明可控。   同时，通过计划书的线上生成，让保险的展业过程变的更简单。保险计划书的精细化运营和可优化的功能实现也让保险变得可定制与人性化。有了互联网的加持，祁安保险经纪APP移动端即可实现协助经纪人理赔、保全、下载电子保单等服务，从而大大提升效率，让保险变得更保险。陈剑峰还在发言中阐释了大家比较关心的分佣机制，其优于同行业的收入提奖比例获得现场的阵阵掌声。接着，祁安保险经纪运营总监郑尊义对核心业务进行了详尽的说明并做了系统的岗前培训。   祁安保险经纪运营总监郑尊义介绍核心业务 在下午的机构交流环节，各分公司负责人在从业经验以及目前遇到的经营问题等方面进行了心得分享，并对未来的工作提出了各自的想法与建议。随后，山东分公司、黑龙江分公司、河北分公司、杭州第一分公司、杭州第二分公司、杭州第三分公司共六家分公司在现场与祁安保险经纪常务副总罗建灿签署了年度业绩承诺书。   祁安保险经纪常务副总罗建灿、祁安保险经纪常务副总陈剑峰与签署年度业绩承诺书的六家分公司代表合影 祁安保险经纪常务副总罗建灿最后也表示将会积极服务好各地分公司，针对大家提出的费用政策、财务支持等方面的诉求将会综合考量并给予最大力度的支持。也希望各地分公司能够积极开拓业务，力争成为全国性保险服务机构的领先者，打造行业联盟式保险服务集团。",
                               "B":"长按扫描二维码 下载“恒实信用”APP 企业信息，尽在掌握！ 为贯彻落实党中央决策部署和6月20日国务院常务会议精神，经国务院同意，人民银行、银保监会、证监会、发展改革委、财政部联合印发《关于进一步深化小微企业金融服务的意见》（银发〔2018〕162号，以下简称《意见》）。《意见》从货币政策、监管考核、内部管理、财税激励、优化环境等方面提出23条短期精准发力、长期标本兼治的具体措施，督促和引导金融机构加大对小微企业的金融支持力度，缓解小微企业融资难融资贵，切实降低企业成本，促进经济转型升级和新旧动能转换。 　　加大货币政策支持力度，引导金融机构聚焦单户授信500万元及以下小微企业信贷投放。《意见》提出，一是增加支小支农再贷款和再贴现额度共1500亿元，下调支小再贷款利率0.5个百分点。二是完善小微企业金融债券发行管理，支持银行业金融机构发行小微企业贷款资产支持证券，盘活信贷资源1000亿元以上。三是将单户授信500万元及以下的小微企业贷款纳入中期借贷便利（MLF）的合格抵押品范围。改进宏观审慎评估体系，增加小微企业贷款考核权重。 　　加大财税政策激励，提高金融机构支小积极性。《意见》提出，一是从2018年9月1日至2020年底，将符合条件的小微企业和个体工商户贷款利息收入免征增值税单户授信额度上限，由100万元提高到500万元。二是对国家融资担保基金支持的融资担保公司加强监管，支持小微企业融资的担保金额占比不低于80%，其中支持单户授信500万元及以下小微企业贷款及个体工商户、小微企业主经营性贷款的担保金额占比不低于50%，适当降低担保费率和反担保要求。 　　加强贷款成本和贷款投放监测考核，促进企业成本明显降低。《意见》强调，一是银行业金融机构要努力实现单户授信总额1000万元及以下小微企业贷款同比增速高于各项贷款同比增速，有贷款余额的户数高于上年同期水平。二是进一步缩短融资链条，清理不必要的“通道”和“过桥”环节，禁止向小微企业贷款收取承诺费、资金管理费，严格限制收取财务顾问费、咨询费。三是改进信贷政策导向效果评估，着力提高金融机构支持小微企业的精准度。 　　健全普惠金融组织体系，提高服务小微企业的能力和水平。《意见》要求，一是大型银行要继续深化普惠金融事业部建设，向基层延伸普惠金融服务机构网点；鼓励未设立普惠金融事业部的银行增设社区、小微支行。二是推进民营银行常态化设立，引导地方性法人银行业金融机构继续下沉经营管理和服务重心。三是银行业金融机构要强化内部激励，大中型银行要加大内部资金支持力度。深化落实小微企业授信尽职免责办法。四是要运用现代金融科技等手段，推进小微企业应收账款融资专项行动，发挥保险增信分险功能，提高小微企业金融服务可得性。 　　大力拓宽多元化融资渠道，优化营商环境，严厉打击骗贷骗补等违法违规行为。《意见》提出，一是支持发展创业投资和天使投资，完善创业投资、天使投资退出机制；持续深化新三板分层、交易制度改革，完善差异化的发行、信息披露等制度。规范发展区域性股权市场。二是引导小微企业聚焦主业，健全财务制度，守法诚信经营，提升自身信用水平。三是推动建立联合激励和惩戒机制，依法依规查处小微企业和金融机构内外勾结、弄虚作假、骗贷骗补等违法违规行为，确保政策真正惠及小微企业。 　　小微活，就业旺，经济兴，做好小微企业金融服务责任重大。金融部门要认真贯彻党中央、国务院部署，切实转变观念，按照《意见》提出的各项要求，不断把金融服务小微企业工作推向深入，为服务小微企业蓬勃发展、促进创新创业与新动能壮大做出应有的贡献！ *声明：本公司业务范围覆盖全国，除保定办事处外，公司全国范围内暂无任何分支机构，出具的关于征信的所有企业资料均以在本公司官网公示为准。其他任何以我公司名义经营谋利的不法个人和组织，我公司将依法追究法律责任。"},
                              {"id":6,
                               "A":"白云机场今年旅客吞吐量有望突破7000万人次·泰凌医药富马酸奎硫平片人体生物等效性试验项目一致性评价进展顺利·“入摩”名单敲定 A股反弹或继续·户用光伏企业联名建言不应被“一刀切” 业内称离平价上网还有一定距离·捷顺科技称股价“非理性下跌” 拟最多3亿元回购股票湖北广电：对本次可转债发行具有信心·超频三：重组拟全资控股炯达能源 继续停牌·舒华体育IPO：与安踏体育关联交易秘而不宣·萃华珠宝拟募投项目现场萧条 国际珠宝城被输血4年前景堪忧·美团缄默应对赴港IPO传言 有投资者担忧其急速扩张地方融资平台资管计划延期再添一例 融资方澄清非兑付危机系期限调整开拓中日经贸合作的新领域美股收评：华尔街三大股指收跌，小盘股依然强势郭思治：大市在低成交下续反复不明　　中国银保监会28日一天连发6份监管函，包括紫金财产保险股份有限公司、安诚财产保险股份有限公司、民生通惠资产管理有限公司、众安在线财产保险股份有限公司、幸福人寿保险股份有限公司、中华联合财产保险股份有限公司皆被点名，并责令其进行限时整改。从这6份监管函来看，本次监管聚焦保险资金的运用情况，部分保险公司存在着资金运用制度不健全、股权投资运作不规范、未按规定进行监管报告和对外信息披露等问题。(小编：散户吧)",
                               "B":"易纲：三季度开始全国性银行业金融机构要将小微企业金融服务情况报央行和银保监 2018年06月30日 22:56:56央行行长易纲29日表示，从2018年三季度开始，全国性银行业金融机构要在季后20日内，将小微企业金融服务的情况、成效、问题、下一步打算以及有关政策建议报人民银行和银保监会，各省(市、区)辖内金融机构要将有关情况报当地人民银行分支机构和银保监会派出机构。（证券时报）"}]}

In [7]:
pos_samples

{'pos_samples': [{'A': '银保监会：引导村镇银行的设立重点向中西部地区倾斜。',
   'B': '筹码高位分散，调整早有预示//@中欧苹果:平安最近不安',
   'id': 1},
  {'A': '新华网北京7月17日消息，财政部17日晚连发4份通报，公布云南、广西、安徽、宁波等地政府违法违规举债担保问责案例www.huijindi.com。相关地方政府及时问责违法违规举债担保行为，对防范地方债务风险具有积极意义。\u3000\u3000涉及云南省的通报点名保山市、昆明市宜良县、楚雄州禄丰县、普洱市景东县等4个地方违法违规举债担保。以昆明市宜良县为例，2016年8月，宜良县金汇国有资产经营有限责任公司与光大兴陇信托有限责任公司签订信托融资协议，计划融资金额5亿元h+u+i+j+i+n+d+i+c+o+m。宜良县人大常委会出具决议，承诺将该笔融资资金列入县本级财政公共预算，按时足额偿还贷款本息；宜良县人民政府签订了关于“财产信托标的债权”的《债券债务确认协议》，承担无条件和不可撤销的标的债权支付义务；宜良县财政局出具将相关偿付“财产权信托标的债权”列入财政预算和中期财政预算的函。截至2017年2月底，该笔融资到位4.24亿元。整改期间，相关承诺函被撤回来自huijindi.com。\u3000\u3000涉及广西壮族自治区的通报点名来宾市、贺州市两起违法违规举债担保案例。记者注意到，上述两地均以市政道路、防洪堤、排洪河桥等公益性资产作为租赁物进行融资。整改期间，来宾市和贺州市政府都经协商有关方面后按程序撤回相关函件，分别将已到账的7亿元和6亿元资金退回有关方面汇_金_网。\u3000\u3000涉及安徽省的通报指出池州经济技术开发区以政府购买服务名义，支持企业通过应收账款抵质押融资实际替政府举债的行为。作为整改措施，池州经济技术开发区解除了相关政府购买服务协议、协商撤销《应收账款质押协议》及相关融资要件、全部归还了5亿元银行贷款。另外，交通银行池州分行由于违法违规向地方政府提供融资并要求地方政府违法违规提供担保，也受到银保监会池州银监分局相应处罚欢迎huijindi.com。\u3000\u3000涉及宁波市的通报指出鄞州区财政局与鄞州城投签订回购协议以建设-移交(BT)方式举债的行为。2017年5月4日，鄞州区财政局、鄞州区城投公司终止了

# 预处理

- 4、对AAB分词、用textrank提取关键词
- 5、对分词后的结果，用TFIDF、卡方提取特征

In [9]:
from jieba import analyse
# 引入TextRank关键词抽取接口
textrank = analyse.textrank

In [8]:
pos_content_list = [pos_con['A'] + '。' + pos_con['B'] for pos_con in pos_samples['pos_samples']]

In [26]:
keyword_list = [textrank(text) for text in pos_content_list]
print('文本数量：', len(keyword_list))
print('每个文本keyword个数：', [len(key_list) for key_list in keyword_list])
for key in keyword_list:
    print(key)

文本数量： 3
每个文本keyword个数： [12, 20, 20]
['筹码', '调整', '重点', '高位', '地区', '倾斜', '中欧', '设立', '分散', '预示', '引导', '苹果']
['政府', '相关', '理赔', '融资', '协议', '通报', '客户', '分公司', '地方', '服务', '宜良县', '举债', '泰康人寿', '担保', '进行', '池州', '风险', '衡阳', '湖南', '债权']
['产品', '客户', '计划', '需求', '保险', '车辆', '中邮', '保单', '受损', '公司', '满足', '微信', '收益', '拒赔', '查询', '功能', '体验', '分公司', '信息', '深入']


In [27]:
keyword_list = [textrank(text) for text in pos_content_pre]
print('文本数量：', len(keyword_list))
print('每个文本keyword个数：', [len(key_list) for key_list in keyword_list])
for key in keyword_list:
    print(key)

文本数量： 3
每个文本keyword个数： [9, 20, 20]
['筹码', '高位', '地区', '设立', '倾斜', '分散', '重点', '调整', '引导']
['政府', '通报', '理赔', '相关', '协议', '融资', '服务', '客户', '分公司', '地方', '举债', '宜良县', '进行', '泰康人寿', '池州', '担保', '债权', '标的', '债务', '信托']
['产品', '计划', '客户', '中邮', '需求', '车辆', '拒赔', '功能', '部分', '保险', '具有', '深入', '保费', '转型', '信息', '查询', '导向', '调研', '保单', '了解']


In [71]:
# keyword_list = textrank(pos_content_pre)
keyword_list

[['筹码', '高位', '地区', '设立', '倾斜', '分散', '重点', '调整', '引导'],
 ['政府',
  '通报',
  '理赔',
  '相关',
  '协议',
  '融资',
  '服务',
  '客户',
  '分公司',
  '地方',
  '举债',
  '宜良县',
  '进行',
  '泰康人寿',
  '池州',
  '担保',
  '债权',
  '标的',
  '债务',
  '信托'],
 ['产品',
  '计划',
  '客户',
  '中邮',
  '需求',
  '车辆',
  '拒赔',
  '功能',
  '部分',
  '保险',
  '具有',
  '深入',
  '保费',
  '转型',
  '信息',
  '查询',
  '导向',
  '调研',
  '保单',
  '了解']]

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer

In [34]:
pos_content_list[0]

'银保监会：引导村镇银行的设立重点向中西部地区倾斜。。筹码高位分散，调整早有预示//@中欧苹果:平安最近不安'

In [68]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(pos_content_pre)
tf_transformer = TfidfTransformer(norm=None, use_idf=False).fit_transform(X_train_counts)
tf_transformer.shape
tf_transformer[0].data

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [69]:
count_vect.get_feature_names()

['bt',
 'com',
 'huijindi',
 'www',
 '一直',
 '一起',
 '上市',
 '上述',
 '不可',
 '不安',
 '不足',
 '专人',
 '专项资金',
 '业务',
 '业管部',
 '两地',
 '两起',
 '严格',
 '严重',
 '中午',
 '中国',
 '中心',
 '中期',
 '中西部',
 '中邮',
 '中邮人寿保险股份有限公司',
 '中邮保险',
 '中邮年',
 '为什么',
 '为例',
 '主力',
 '丽为',
 '举一反三',
 '举债',
 '举措',
 '义务',
 '书面',
 '买入',
 '了解',
 '事发',
 '事故',
 '云南',
 '云南省',
 '交通事故',
 '交通银行',
 '产品',
 '人大常委会',
 '人民政府',
 '亿元',
 '价值',
 '企业',
 '传美图',
 '伤害',
 '体验',
 '作出',
 '保单',
 '保山市',
 '保费',
 '保险',
 '保险产品',
 '保险代理',
 '保障',
 '信号',
 '信心',
 '信息',
 '信托',
 '债券',
 '债务',
 '债权',
 '倾斜',
 '做出',
 '偿付',
 '偿还',
 '充分',
 '先行',
 '光大兴陇信托',
 '党内',
 '入市',
 '全力以赴',
 '全方位',
 '全省',
 '全部',
 '公共',
 '公司',
 '公布',
 '公益性',
 '关于',
 '具有',
 '决议',
 '减免',
 '减税',
 '出具',
 '出险',
 '函件',
 '分公司',
 '分别',
 '分散',
 '分析',
 '分管',
 '分行',
 '分许',
 '列入',
 '利用',
 '到位',
 '制造业',
 '副组长',
 '办理',
 '功能',
 '加快',
 '包含',
 '化解',
 '北京',
 '医疗',
 '医疗保险',
 '医院',
 '半挂车',
 '协商',
 '协议',
 '协调',
 '及时',
 '反应',
 '发展趋势',
 '发生',
 '受伤',
 '受到',
 '受损',
 '受理',
 '可拨打',
 '可能',
 '各线',
 '名义',
 '启动',
 '启用',
 '呵护',
 

In [62]:
X_train_counts[0].data

array([1, 1, 1, 1, 1, 1], dtype=int64)

In [53]:
dir(X_train_counts[0])

['__abs__',
 '__add__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__numpy_ufunc__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmatmul__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_arg_min_or_max',
 '_arg_min_or_max_axis',
 '_binopt',
 '_boolean_index_to_array',
 '_check_boolean',
 '_check_ellipsis',
 '_cs_matrix__get_has_canonical_format',
 '_cs_matrix__get_sorted',
 '_cs_matrix__set_has_ca

In [61]:
tf_transformer[0].data

array([0.40824829, 0.40824829, 0.40824829, 0.40824829, 0.40824829,
       0.40824829])

In [59]:
1/6

0.16666666666666666

In [37]:
help(tf_transformer)

Help on csr_matrix in module scipy.sparse.csr object:

class csr_matrix(scipy.sparse.compressed._cs_matrix, scipy.sparse.sputils.IndexMixin)
 |  Compressed Sparse Row matrix
 |  
 |  This can be instantiated in several ways:
 |      csr_matrix(D)
 |          with a dense matrix or rank-2 ndarray D
 |  
 |      csr_matrix(S)
 |          with another sparse matrix S (equivalent to S.tocsr())
 |  
 |      csr_matrix((M, N), [dtype])
 |          to construct an empty matrix with shape (M, N)
 |          dtype is optional, defaulting to dtype='d'.
 |  
 |      csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
 |          where ``data``, ``row_ind`` and ``col_ind`` satisfy the
 |          relationship ``a[row_ind[k], col_ind[k]] = data[k]``.
 |  
 |      csr_matrix((data, indices, indptr), [shape=(M, N)])
 |          is the standard CSR representation where the column indices for
 |          row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
 |          corresponding value

In [23]:
pos_content_pre = pre_cor.handle_contents(pos_content_list)
print('文本数量：', len(pos_content_pre))
print('每个文本keyword个数：', [len(key_list) for key_list in pos_content_pre])
print(pos_content_pre)
pos_content_pre[0]

文本数量： 3
每个文本keyword个数： [57, 2024, 364]
['银保监 会 引导 村镇银行 设立 重点 中西部 地区 倾斜 筹码 高位 分散 调整 早有 预示 平安 最近 不安 ', '新华网 北京 月 日 消息 财政部 日晚 连发 份 通报 公布 云南 广西 安徽 宁波 政府 违法违规 举债 担保 问责 案例 www huijindi com 相关 地方 政府 及时 问责 违法违规 举债 担保 行为 对 防范 地方 债务 风险 具有 积极意义 涉及 云南省 通报 点名 保山市 昆明市 宜良县 楚雄州 禄丰县 普洱 市 景东 县 地方 违法违规 举债 担保 以 昆明市 宜良县 为例 年 月 宜良县 金汇 国有资产 经营 有限责任 公司 光大兴陇信托 有限责任 公司 签订 信托 融资 协议 计划 融资 金额 亿元 h u i j i n d i c o m 宜良县 人大常委会 出具 决议 承诺 将 该笔 融资 资金 列入 县 本级 财政 公共 预算 按时 足额 偿还 贷款 本息 宜良县 人民政府 签订 关于 财产 信托 标的 债权 债券 债务 确认 协议 承担 无条件 不可 撤销 标的 债权 支付 义务 宜良县 财政局 出具 将 相关 偿付 财产权 信托 标的 债权 列入 财政预算 中期 财政预算 函 截至 年 月底 该笔 融资 到位 亿元 整改 期间 相关 承诺函 被 撤回 来自 huijindi com 涉及 广西壮族自治区 通报 点名 来宾市 贺州市 两起 违法违规 举债 担保 案例 注意 到 上述 两地 均 以 市政道路 防洪堤 排洪 河桥 公益性 资产 租赁 物 进行 融资 整改 期间 来宾市 贺州市 政府 协商 方面 后 按程序 撤回 相关 函件 分别 将 已 到 账 亿元 亿元 资金 退回 方面 汇 金 网 涉及 安徽省 通报 指出 池州 经济 技术开发区 以 政府 购买 服务 名义 支持 企业 通过 应收 账款 抵 质押 融资 实际 替 政府 举债 行为 整改措施 池州 经济 技术开发区 解除 相关 政府 购买 服务 协议 协商 撤销 应收 账款 质押 协议 相关 融资 要件 全部 归还 亿元 银行贷款 交通银行 池州 分行 违法违规 地方 政府 提供 融资 并 要求 地方 政府 违法违规 提供 担保 也 受到 银保监 会 池州 银监

'银保监 会 引导 村镇银行 设立 重点 中西部 地区 倾斜 筹码 高位 分散 调整 早有 预示 平安 最近 不安 '

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Pipeline([
            ('textrank', textrank(text)),
            ('counts', CountVectorizer()),
            ('tf', TfidfTransformer(norm=None, use_idf=False))
        ])),
        ('len_stats', StatsFeatures())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=2))
])

pipeline.fit(X_train, y_train)

In [ ]:
class Statskeywords(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def 

In [ ]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()

    def fit(self, X, y=None):
        return self

    def getcnt(self,x): 
        '''词个数'''
        return len(list(set(x)))

    def getnegcnt(self,x):
        '''负面词个数'''
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt

    def getrepcnt(self,x):
        '''重复词个数'''
        rep_word ={}
        words = x.split()        
        for w in self.getcnt(x):
            if words.count(w)>1: # 记录重复词汇（词频大于1）
                rep_word[w] = words.count(w)
        return len(rep_word)
    
    def transform(self, X):
        '''
        文本长度、词个数、词比例、
        负面词个数、负面词比例、
        重复词个数、重复词比例
        '''
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length,
                         self.getrepcnt(x),self.getrepcnt(x)/length])            
        return data